In [57]:
import os
from getpass import getpass

from dotenv import load_dotenv



In [58]:
import pstuts_rag

In [59]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
from dataclasses import dataclass
@dataclass
class ApplicationParameters:
    filename = "data/test.json"
    embedding_model = "text-embedding-3-small"
    n_context_docs = 2

params = ApplicationParameters()

In [61]:

load_dotenv()

def set_api_key_if_not_present(key_name, prompt_message=""):
    if len(prompt_message) == 0:
        prompt_message=key_name
    if key_name not in os.environ or not os.environ[key_name]:
        os.environ[key_name] = getpass.getpass(prompt_message)

set_api_key_if_not_present("OPENAI_API_KEY")

# Data Preparation

First, we will read in the transcripts of the videos and convert them to Documents
with appropriate metadata.

In [62]:
from ast import Dict
import json
filename = "../data/test.json"
from typing import List, Dict, Any
data:List[Dict[str,Any]] = json.load(open(filename, "rb"))


In [63]:
data

[{'video_id': 4157,
  'title': 'Get organized with layer groups',
  'desc': 'Learn some great tips for working with layers.',
  'length': '00:04:05.78',
  'url': 'https://videos-tv.adobe.com/2013-07-23/f65b5a0ef188ba5e5a96df93a8ead3cf.mp4',
  'transcripts': [{'sent_id': 0,
    'sent': "At any time when you're working in Adobe Photoshop on a complicated subject the possibility exists of chaos ensuing.",
    'begin': 1.27,
    'end': 9.97},
   {'sent_id': 1,
    'sent': "Well, you know what chaos there is, don't you?",
    'begin': 10.16,
    'end': 12.01},
   {'sent_id': 2,
    'sent': "It's my studio in the middle of a project.",
    'begin': 12.009999,
    'end': 14.23},
   {'sent_id': 3,
    'sent': 'What I would like to do is reduce the clutter, reduce the chaos.',
    'begin': 14.58,
    'end': 17.719999},
   {'sent_id': 4,
    'sent': 'Let me move Layers over here again, I would say that is not really necessary, but it makes life easier.',
    'begin': 17.719999,
    'end': 23.15}

## R - retrieval

Let's hit it with a semantic chunker.

In [64]:
from pstuts_rag.rag import RetrieverFactory

retriever_factory = RetrieverFactory()
retriever_factory.add_docs(raw_docs=data)

## A - Augmentation

We need to populate a prompt for LLM.


## Generation

We will use a 4.1-nano to generate answers.

In [66]:
from pstuts_rag.rag import RAGChainFactory

rag_factory = RAGChainFactory(retriever=retriever_factory.get_retriever())


In [65]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini",temperature=0)

In [67]:

get_videos = rag_factory.get_rag_chain(llm)
    


In [68]:
val = get_videos.invoke({"question":"What are layers"})

In [69]:
val.additional_kwargs

{'refusal': None,
 'context': [Document(metadata={'video_id': 19172, 'title': 'Understand layers', 'desc': 'Learn what layers are and why they are so useful.', 'length': '00:04:44.75', 'source': 'https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4', 'speech_start_stop_times': [[0.47, 3.41], [3.81, 9.13], [9.309999, 15.01], [15.299999, 20.57], [20.88, 23.3], [23.83, 27.93], [29.38, 32.79], [32.96, 33.92], [34.43, 40.21], [41.91, 45.37], [45.88, 49.01], [49.54, 55.130001], [55.72, 58.49], [58.72, 62.14]], 'start': 0.47, 'stop': 62.14, '_id': '66a08ee37c2b4626a5a1a410fd1196e1', '_collection_name': '6f5ca406-d69b-4072-b039-f79ca4eab2a9_qdrant'}, page_content="Layers are the building blocks of any image in Photoshop CC. So, it's important to understand, what layers are and why to use them - which we'll cover in this video. If you're following along, open this 

In [70]:
val.pretty_print()

================================== Ai Message ==================================

Layers are the building blocks of any image in Photoshop CC. You can think of layers like separate flat panes of glass stacked on top of each other. Each layer contains separate pieces of content. Some areas of a layer can be transparent, allowing you to see through to the layers below. This setup lets you edit parts of an image independently without affecting the rest of the image. You work with layers in the Layers panel, where you can toggle their visibility on and off using the Eye icon. (See 0:28–1:02 and 1:25–2:32) 🎨🖼️
**References**:
[
  {
    "title": "Understand layers",
    "source": "https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4",
    "start": 0.47,
    "stop": 62.14
  },
  {
    "title": "Understand layers",
    "source": "https://images-tv.adobe.com/avp/vr